In [ ]:
!pip install pyspark
!pip install neo4j
!pip install pandas

import pandas as pd

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ejemplo de PySpark en Jupyter Notebook") \
    .getOrCreate()

# Obtener el SparkContext
sc = spark.sparkContext

In [ ]:
#B = 150

In [ ]:
def hash(x, B):
    return x % B

In [ ]:
def map_pdm(x, arista, y, B):

    x_hash = hash(x, B)
    y_hash = hash(y, B)

    l = []
    for i in range(B):
        l.append(((x_hash, y_hash, i), (x, arista, y)))
        l.append(((i, x_hash, y_hash), (x, arista, y)))
        l.append(((y_hash, i, x_hash), (x, arista, y)))

    return set(l)

In [ ]:
def triangulo(nodos):

  triangulos_detectados = []
  for i in range(len(nodos)):
    nodo_actual = nodos[i]

    for j in range(i + 1, len(nodos)):

      nodo_sig = nodos[j]

      for k in range(j + 1, len(nodos)):
        nodo_sub_sig = nodos[k]

        if nodo_actual[2] == nodo_sig[0] and nodo_sig[2] == nodo_sub_sig[0] and nodo_actual[0] == nodo_sub_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

        elif nodo_actual[2] == nodo_sub_sig[0] and nodo_sig[0] == nodo_sub_sig[2] and nodo_actual[0] == nodo_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

  return triangulos_detectados

In [ ]:
from neo4j import GraphDatabase

URI = "neo4j+s://06ae1fa1.databases.neo4j.io"
AUTH = ("neo4j","QjwVk3kN-OI5bTt-fg6LZy-F4LMHCpL9HFxuvSuq-OE")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

Connection to Neo4j established successfully!


### Función a cambiar

In [ ]:
def get_data_from_neo4j():
    with driver.session() as session:
        result = session.run("""
        MATCH (n1:Node)-[r:RELATED]->(n2:Node)
        RETURN n1.id AS id_form, n2.id AS id_to, r.weight AS weight
        """)
        data = [record.data() for record in result]
        tuples = [(d['id_form'], d['weight'], d['id_to']) for d in data]
    return tuples

# Obtiene los datos de Neo4j
neo4j_data = get_data_from_neo4j()

In [ ]:
rdd = sc.parallelize(neo4j_data)

In [ ]:
rdd.take(5)

[(1, 1, 2), (1, 7, 3), (4, 3, 5), (6, 5, 7), (6, 14137, 8)]

In [ ]:
B = 100

In [ ]:
#map
rdd_neo4j_map = rdd.flatMap(lambda dato: map_pdm(*dato, B))
rdd_neo4j_map.collect()

[((1, 2, 97), (1, 1, 2)),
 ((2, 54, 1), (1, 1, 2)),
 ((31, 1, 2), (1, 1, 2)),
 ((1, 2, 96), (1, 1, 2)),
 ((2, 47, 1), (1, 1, 2)),
 ((7, 1, 2), (1, 1, 2)),
 ((44, 1, 2), (1, 1, 2)),
 ((2, 31, 1), (1, 1, 2)),
 ((69, 1, 2), (1, 1, 2)),
 ((2, 25, 1), (1, 1, 2)),
 ((51, 1, 2), (1, 1, 2)),
 ((2, 76, 1), (1, 1, 2)),
 ((30, 1, 2), (1, 1, 2)),
 ((2, 79, 1), (1, 1, 2)),
 ((2, 14, 1), (1, 1, 2)),
 ((27, 1, 2), (1, 1, 2)),
 ((1, 2, 44), (1, 1, 2)),
 ((58, 1, 2), (1, 1, 2)),
 ((1, 2, 5), (1, 1, 2)),
 ((2, 68, 1), (1, 1, 2)),
 ((1, 2, 12), (1, 1, 2)),
 ((1, 2, 81), (1, 1, 2)),
 ((90, 1, 2), (1, 1, 2)),
 ((2, 11, 1), (1, 1, 2)),
 ((94, 1, 2), (1, 1, 2)),
 ((2, 5, 1), (1, 1, 2)),
 ((1, 2, 32), (1, 1, 2)),
 ((1, 2, 63), (1, 1, 2)),
 ((1, 2, 98), (1, 1, 2)),
 ((1, 2, 7), (1, 1, 2)),
 ((26, 1, 2), (1, 1, 2)),
 ((2, 32, 1), (1, 1, 2)),
 ((2, 35, 1), (1, 1, 2)),
 ((2, 71, 1), (1, 1, 2)),
 ((35, 1, 2), (1, 1, 2)),
 ((3, 1, 2), (1, 1, 2)),
 ((64, 1, 2), (1, 1, 2)),
 ((32, 1, 2), (1, 1, 2)),
 ((34, 1, 2), (1,

In [ ]:
reduce_neo4j = rdd_neo4j_map.groupByKey().mapValues(list)
reduce_neo4j.take(10)

[((1, 2, 10),
  [(1, 1, 2),
   (2, 2, 310),
   (610, 1, 101),
   (10, 1, 901),
   (10, 5, 101),
   (501, 2, 402),
   (401, 6, 402),
   (301, 11, 2),
   (701, 164, 702),
   (410, 145, 701),
   (702, 61, 410),
   (101, 153, 102),
   (102, 27, 10)]),
 ((1, 2, 20),
  [(1, 1, 2),
   (501, 2, 402),
   (401, 6, 402),
   (301, 11, 2),
   (701, 164, 702),
   (101, 153, 102),
   (202, 82, 220)]),
 ((1, 3, 91),
  [(1, 7, 3),
   (701, 145, 503),
   (701, 9, 403),
   (701, 8, 703),
   (703, 5, 1191),
   (203, 76, 591),
   (203, 35915, 191),
   (103, 1425, 191)]),
 ((3, 95, 1),
  [(1, 7, 3),
   (701, 145, 503),
   (701, 9, 403),
   (701, 8, 703),
   (95, 175, 101),
   (195, 49476, 1401),
   (203, 108, 95),
   (203, 2323, 195),
   (103, 289, 195)]),
 ((66, 6, 7),
  [(6, 5, 7),
   (1006, 353, 1007),
   (1206, 80, 807),
   (806, 156, 807),
   (466, 32, 106),
   (266, 4, 106),
   (206, 1, 707)]),
 ((5, 66, 12),
  [(12, 413, 5),
   (66, 42, 212),
   (5, 2, 1266),
   (466, 2, 212),
   (105, 36, 166),
   (

In [ ]:
trangulos = reduce_neo4j.map(lambda nodos: (nodos[0], triangulo(nodos[1])))
trangulos.take(10)

[((56, 19, 24), []),
 ((98, 95, 24), [(124, 195, 198)]),
 ((70, 46, 45), [(245, 46, 70)]),
 ((83, 82, 46),
  [(246, 82, 83),
   (46, 82, 83),
   (46, 83, 182),
   (46, 82, 183),
   (46, 182, 183),
   (546, 82, 83),
   (546, 82, 183),
   (146, 82, 83),
   (146, 83, 182),
   (146, 82, 183),
   (146, 182, 183)]),
 ((39, 17, 11), []),
 ((96, 81, 78), [(81, 178, 196)]),
 ((21, 85, 89), []),
 ((81, 94, 46),
  [(46, 81, 194), (46, 181, 194), (146, 81, 194), (146, 181, 194)]),
 ((14, 31, 62), []),
 ((71, 62, 80),
  [(62, 80, 171), (62, 80, 271), (62, 171, 180), (62, 180, 271)]),
 ((97, 97, 67), []),
 ((50, 49, 40),
  [(49, 250, 240),
   (49, 150, 240),
   (49, 350, 240),
   (249, 150, 240),
   (249, 350, 240)]),
 ((80, 98, 83), []),
 ((85, 35, 95), []),
 ((52, 57, 88),
  [(52, 57, 88),
   (52, 57, 188),
   (52, 157, 88),
   (52, 157, 188),
   (252, 257, 88),
   (57, 152, 88),
   (57, 88, 352),
   (57, 152, 188),
   (57, 188, 352),
   (152, 157, 88),
   (152, 157, 188)]),
 ((58, 55, 52), [(952,

In [ ]:
d = {}
for i in trangulos.collect():
    d[i[0]] = i[1]

In [ ]:
def buscar_triangulos(rdd, b):
    rdd_neo4j_map = rdd.flatMap(lambda dato: map_pdm(*dato, b))
    reduce_neo4j = rdd_neo4j_map.groupByKey().mapValues(list)
    trangulos = reduce_neo4j.map(lambda nodos: (nodos[0], triangulo(nodos[1])))

    dicc = {}
    for i in trangulos.collect():
        dicc[i[0]] = i[1]

    return dicc



In [ ]:
test = sc.parallelize(neo4j_data)

buscar_triangulos(test, 100)

{(89, 1, 3): [],
 (6, 8, 23): [],
 (14, 80, 13): [],
 (20, 22, 79): [],
 (29, 5, 67): [],
 (30, 31, 66): [],
 (47, 30, 34): [(147, 1034, 730)],
 (90, 44, 45): [],
 (58, 97, 44): [],
 (62, 79, 44): [(44, 62, 79)],
 (72, 75, 44): [],
 (44, 75, 94): [],
 (76, 51, 44): [],
 (44, 78, 21): [],
 (44, 79, 24): [],
 (76, 44, 85): [(44, 85, 376)],
 (92, 27, 44): [],
 (44, 94, 61): [],
 (76, 44, 95): [(44, 95, 376)],
 (44, 5, 4): [],
 (44, 5, 14): [(44, 314, 105)],
 (7, 8, 10): [(407, 408, 410)],
 (67, 7, 11): [],
 (26, 24, 29): [],
 (24, 49, 98): [],
 (24, 49, 88): [(124, 49, 88), (124, 49, 188)],
 (32, 24, 31): [(124, 131, 132)],
 (52, 24, 47): [(124, 147, 152)],
 (24, 75, 10): [],
 (24, 97, 58): [(124, 158, 197)],
 (85, 24, 4): [(124, 185, 204)],
 (15, 69, 17): [],
 (23, 72, 16): [],
 (90, 13, 16): [],
 (95, 16, 90): [],
 (33, 39, 31): [(339, 731, 733)],
 (41, 96, 76): [],
 (74, 45, 46): [(245, 46, 174), (245, 46, 274), (46, 145, 174)],
 (74, 45, 56): [(245, 56, 174), (245, 56, 274)],
 (45, 62

In [ ]:
driver.close()